In [1]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)


In [13]:

import pandas as pd
from sklearn.model_selection import train_test_split
import os

In [14]:
# # Load CSV
# df = pd.read_csv("E:/deepfake/media/processeddata/train_labels.csv")

# # Encode labels: real -> 0, fake -> 1
# df['label'] = df['label'].map({'real': 0, 'fake': 1})

# # Split into train and validation
# train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# print("Train:", len(train_df), "Validation:", len(val_df))
df = pd.read_csv("E:/deepfake/media/processeddata/train_labels.csv")

# Encode labels
df['label'] = df['label'].map({'real': 0, 'fake': 1})

# Train/validation split
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

print("Train samples:", len(train_df))
print("Validation samples:", len(val_df))


Train samples: 42515
Validation samples: 10629


In [15]:
# Map back integer labels to string labels
train_df['label'] = train_df['label'].map({0: 'real', 1: 'fake'})
val_df['label'] = val_df['label'].map({0: 'real', 1: 'fake'})


# It is option 1 for equal number of datasets

In [16]:
# It is option to for imbalance data set 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [17]:
# Image generator with rescaling
datagen = ImageDataGenerator(rescale=1./255)

# Directory path for generator (base_dir = E:/deepfake_processed)
base_dir = "E:/deepfake/media/processeddata"

train_gen = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=base_dir,
    x_col='filename',
    y_col='label',
    target_size=(224, 224),
    class_mode='binary',
    batch_size=32,
    shuffle=True
)

val_gen = datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=base_dir,
    x_col='filename',
    y_col='label',
    target_size=(224, 224),
    class_mode='binary',
    batch_size=32,
    shuffle=False
)

Found 42515 validated image filenames belonging to 2 classes.
Found 10629 validated image filenames belonging to 2 classes.


In [18]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

model = Sequential([
    Input(shape=(224, 224, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 186624)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 128)                 │      23,888,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,907,521 (91.20 MB)

 Trainable params: 23,907,521 (91.20 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15  # Train more if needed
)


E:\deepfake\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15


E:\deepfake\venv\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1329/1329 ━━━━━━━━━━━━━━━━━━━━ 3061s 2s/step - accuracy: 0.5884 - loss: 0.7245 - val_accuracy: 0.6535 - val_loss: 0.6045
Epoch 2/15
1329/1329 ━━━━━━━━━━━━━━━━━━━━ 2851s 2s/step - accuracy: 0.6728 - loss: 0.5866 - val_accuracy: 0.6709 - val_loss: 0.5833
Epoch 3/15
1329/1329 ━━━━━━━━━━━━━━━━━━━━ 2750s 2s/step - accuracy: 0.7011 - loss: 0.5530 - val_accuracy: 0.6786 - val_loss: 0.5707
Epoch 4/15
 432/1329 ━━━━━━━━━━━━━━━━━━━━ 29:36 2s/step - accuracy: 0.7279 - loss: 0.5156

In [5]:
model.save("E:/deepfake/models/deepfake_detector_partial_epoch1.h5")


In [25]:
from tensorflow.keras.models import load_model
model = load_model("E:/deepfake/models/deepfake_detector_partial_epoch1.h5")


In [26]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [27]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5,
    class_weight=class_weights
)


E:\deepfake\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
E:\deepfake\venv\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/5
2147/2147 ━━━━━━━━━━━━━━━━━━━━ 5505s 3s/step - accuracy: 0.8613 - loss: 0.4423 - val_accuracy: 0.8875 - val_loss: 0.4217
Epoch 2/5
2147/2147 ━━━━━━━━━━━━━━━━━━━━ 4663s 2s/step - accuracy: 0.8949 - loss: 0.2839 - val_accuracy: 0.9020 - val_loss: 0.3741
Epoch 3/5
2147/2147 ━━━━━━━━━━━━━━━━━━━━ 4690s 2s/step - accuracy: 0.9059 - loss: 0.2603 - val_accuracy: 0.9059 - val_loss: 0.3883
Epoch 4/5
2147/2147 ━━━━━━━━━━━━━━━━━━━━ 4294s 2s/step - accuracy: 0.9079 - loss: 0.2533 - val_accuracy: 0.9092 - val_loss: 0.3674
Epoch 5/5
2147/2147 ━━━━━━━━━━━━━━━━━━━━ 4513s 2s/step - accuracy: 0.9116 - loss: 0.2452 - val_accuracy: 0.9098 - val_loss: 0.4105


In [28]:
model.save("E:/deepfake/models/deepfake_detector_partial_epoch1.h5")